# Deploying Logic Apps
Now that we've verified that our Docker image works, this notebook will walk through how to operationalize this workflow using [Logic Apps](https://azure.microsoft.com/en-us/services/logic-apps/). 

For this workflow, adding a file whose name begins with 'trigger' will kick-off the entire batch style transfer process. In this architecture, we use Logic App as our trigger mechanism: when a trigger file (ex. 'trigger.txt') is uploaded into blob storage, Logic App will detect that the file has been added, and create an [Azure Container Instance (ACI)](https://azure.microsoft.com/en-us/services/container-instances/) to start creating Batch AI jobs.

This notebook will rely on the file __template.logic_app.json__ to deploy the Logic App. This file is a template for a [Azure Resource Manager (ARM)](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-overview) json file, which we will use in this notebook to deploy the resources. In its current state, this file does not contain any of the parameters, such as those stored in the .env file, that we've set throughout the tutorial. To fill out those parameters, we will use [jinja](http://jinja.pocoo.org/) to populate the template with all the appropriate parameters and write the output to __logic_app.json__. This will be ARM json file that we use with the __az cli__ to deploy the Logic App.

As part of the Logic App deployment, we'll also deploy 2 additional resources: an Azure blob storage connector and an ACI connector. These connectors will allow Logic App to talk to these two resources.

---

Import utilities and load environment variables. In this notebook, we'll be using jinja2 to populate our Logic App deployment.

In [1]:
import json
import os
import jinja2
%load_ext dotenv
%dotenv

Set variables we'll need for Logic App deployment:

- `logic_app` - the name of your logic app
- `aci_container_group` - the name of your ACI 
- `aci_display_name` - a display name used for your ACI deployment

In [2]:
logic_app = "<my-logic-app>"
aci_container_group = "<my-aci-group>"
aci_display_name = "<your@email.com>"

Persist these variables to our .env file

In [3]:
!dotenv set LOGIC_APP $logic_app
!dotenv set ACI_CONTAINER_GROUP $aci_container_group
!dotenv set ACI_DISPLAY_NAME $aci_display_name

/bin/sh: 1: Syntax error: end of file unexpected
/bin/sh: 1: Syntax error: end of file unexpected
/bin/sh: 1: Syntax error: end of file unexpected


Reload our environment variables from our .env file.

In [4]:
%reload_ext dotenv
%dotenv

Using Jinja, populate the `template.logic_app.json` file and output the new file as `logic_app.json`. This file will be saved in the working directory. 

In [ ]:
# use jinja to fill in variables from .env file
env = jinja2.Environment(
    loader=jinja2.FileSystemLoader('.')
)
template = env.get_template('template.logic_app.json')

e = os.environ
rendered_template = template.render(env=e)

out = open('logic_app.json', 'w')
out.write(rendered_template)
out.close()

Now that we have the `logic_app.json` file, we can use the __az cli__ to deploy the resource.

In [ ]:
%%bash 
az group deployment create \
    --name $LOGIC_APP \
    --resource-group $RESOURCE_GROUP \
    --template-file logic_app.json